<a href="https://colab.research.google.com/github/chrisdev93c/retos-programacion-2023/blob/main/scrapertwitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install snscrape


scrapear tweets


In [ ]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import re

# Máximo número de tweets a extraer
max_tweets = 10

# Patrones regex para detectar enlaces y hashtags
pattern_links = r"https?://[^\s]+"
pattern_hashtags = r"#\S+"
pattern_emojis = r"([\U00002600-\U000027BF])|([\U0001f300-\U0001f64F])|([\U0001f680-\U0001f6FF])"
pattern_kingsleague = r"\bKingsLeague\b"
pattern_mentions = r"@\S+"
pattern_special_chars = r"[^\w\s]"



# Crea una lista vacía para almacenar los tweets
tweets = []

# Realiza la búsqueda de tweets con el hashtag "KingsLeague" en un rango de fechas
for tweet in sntwitter.TwitterSearchScraper(
    '#kingsleague lang:es since:2022-12-27 until:2023-01-01'
).get_items():
    # Si se ha alcanzado el límite máximo de tweets, detiene el bucle
    if len(tweets) > max_tweets:
        break
    
    # Limpia el texto del tweet eliminando enlaces, hashtags y menciones
    text = tweet.content
    text = re.sub(pattern_links, "", text)
    text = re.sub(pattern_hashtags, "", text)
    text = re.sub(r"@\S+", "", text)
    text = re.sub(pattern_emojis, "", text)
    text = re.sub(pattern_kingsleague, "", text)
    text = re.sub(pattern_mentions, "", text)
    text = re.sub(pattern_special_chars, "", text)




    # Añade el tweet a la lista
    tweets.append({
        "text": text,
      
        
    })

# Convierte la lista de tweets en un dataframe de Pandas
data = pd.DataFrame(tweets)
data.to_csv("tweetss.csv", index=False)

print(data)


limpiamos el texto para las palabras mas repetidas

In [ ]:
import csv
import re

def clean_cell(cell):
    # Elimina artículos y preposiciones
    cell = re.sub(r"\b(El|la|los|las|un|una|unos|unas|del|al)\b", "", cell)
    # Elimina espacios en blanco adicionales
    cell = re.sub(r"\s+", " ", cell).strip()
    return cell

# Abre el archivo CSV original y el archivo CSV de destino
with open("tweets.csv", "r") as fin, open("tweets_clean.csv", "w") as fout:
    # Crea un objeto CSV reader y un objeto CSV writer
    reader = csv.reader(fin)
    writer = csv.writer(fout)
    # Recorre las filas del archivo CSV original
    for row in reader:
        # Limpia cada celda de la fila
        row = [clean_cell(cell) for cell in row]
        # Escribe la fila limpia en el archivo CSV de destino
        writer.writerow(row)


In [ ]:
import pandas as pd
import csv



data = pd.read_csv("tweets.csv")
texts = data["text"].tolist()
import nltk
from collections import Counter



# Separa el texto en una lista de palabras o frases
words = [text.split() for text in texts]

# Cuenta la frecuencia de cada palabra o frase
counter = Counter([word for text in words for word in text])

# Ordena las palabras o frases por frecuencia
most_common = counter.most_common()

# Crea un dataframe de Pandas con las palabras o frases más comunes
df = pd.DataFrame(most_common, columns=["palabra_o_frase", "frecuencia"])

# Guarda el dataframe en un archivo CSV
df.to_csv("palabras_o_frases_mas_comune.csv", index=False)



In [ ]:
!pip install textblob

In [ ]:
import textblob
import pandas as pd
from textblob import TextBlob

# Abre el archivo CSV y carga los datos en un dataframe
data = pd.read_csv("tweets.csv")

# Itera sobre las filas del archivo CSV
for index, row in data.iterrows():
    # Traduce el texto al inglés
    #translation = TextBlob(row["text"]).translate(from_lang='es', to='en')
    #print(translation)
    text = TextBlob(row["text"]).translate(from_lang='es', to='en').raw

    # Obtiene el texto traducido
    #text = translation.text
    # Crea un objeto TextBlob a partir del texto traducido
    blob = TextBlob(text)
    # Obtiene el análisis de sentimientos
    sentiment = blob.sentiment
    # Clasifica el sentimiento como negativo, positivo o neutro
    if sentiment.polarity < 0:
        sentimiento = "negativo"
    elif sentiment.polarity > 0:
        sentimiento = "positivo"
    else:
        sentimiento = "neutro"
    # Añade el sentimiento al dataframe
    data.at[index, "sentimiento"] = sentimiento
  
    data.to_csv("tweets.csv", index=False)